In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from tqdm import tqdm
%matplotlib inline

In [34]:
# Sorted by Precursor m/z
def sort_peaks2(batch_path):
    sample_df = pd.DataFrame()
    num_files = len([f for f in os.listdir(batch_path)if os.path.isfile(os.path.join(batch_path, f))])
    for f in range(num_files-1):
        standard = pd.read_csv(batch_path + os.listdir(batch_path)[f+1], sep="\\t", engine='python')
        standard['samples'] = os.listdir(batch_path)[f+1]
        sample_df = sample_df.append(standard)
    sorted_df = sample_df.sort_values(by=['Precursor m/z'])
    
    peak_num = 0
    listing = []
    for x in range(len(sorted_df)):
        if ((abs(sorted_df.iat[x,4]-sorted_df.iat[(x-1),4]) <= 0.05) & (abs(sorted_df.iat[x,6]-sorted_df.iat[(x-1),6]) <= 0.002)):
            listing.append(peak_num)
        else:
            peak_num = peak_num + 1
            listing.append(peak_num)
    sorted_df['unique peak'] = listing
    return sorted_df.groupby(['unique peak']).mean()

In [47]:
sorted_peaks = sort_peaks2('/mnt/c/Users/nozom/Desktop/mass-suite/example_data/batch/')
reference1 = sorted_peaks[['RT (min)','Precursor m/z', 'Area']]
reference1

,RT (min),Precursor m/z,Area
unique peak,,,
1,21.084500,102.9691,15183.334961
2,0.689925,104.1072,58451.677734
3,18.046520,104.9924,403594.875000
4,19.097570,104.9925,416176.062500
5,0.631683,104.9926,648919.937500
...,...,...,...
1617,19.197180,1364.8640,6554.185059
1618,16.756830,1371.0000,16352.015625
1619,16.735030,1371.0080,9069.611328


In [51]:
def stack(batch_path):
    all_samples = []
    num_files = len([f for f in os.listdir(batch_path)if os.path.isfile(os.path.join(batch_path, f))])
    for files in tqdm(range(num_files)):
        sample = os.listdir(batch_path)[files]
        sample_df = pd.read_csv(batch_path + sample, sep="\\t", usecols=['RT (min)', 'Precursor m/z', 'Area'], dtype=np.float32, engine='python')
        # Prefilter
        #sample_df = sample_df.drop(sample_df[(sample_df['Area'] > 5000) & (sample_df['RT (min)'] < 1.5) & (sample_df['S/N'] > 5)].index)
        all_samples += [sample_df]
    total_samples = pd.concat(all_samples)
    all_samples.clear()
    return total_samples

In [88]:
"""This function works by using one .txt file as a reference in which
other files realigned to in terms of precursor and RT. """
RT_error = 0.10 # units of minutes, can be adjusted
alignment_df = pd.DataFrame()
standard_sample = os.listdir(batch_path)[0] # first sample
# reads .txt file into a dataframe
# standard_df = pd.read_csv(batch_path + standard_sample, sep="\\t", usecols=['RT (min)', 'Precursor m/z'], dtype= np.float32, engine='python')
standard_df = reference1
alignment_df['RT (min)'] = standard_df.iloc[:,0]
alignment_df['Precursor m/z'] = standard_df.iloc[:,1]
alignment_df['Sum RT (min)'] = 0.0
alignment_df['Sum Precursor m/z'] = 0.0
mz_error = 0.00002*standard_df.iloc[:,1] # 20 ppm
col_index = 4
total_samp = stack(batch_path)
num_files = len([f for f in os.listdir(batch_path)if os.path.isfile(os.path.join(batch_path, f))]) 
for files in range(num_files):
    sample = os.listdir(batch_path)[files]
    alignment_df[sample] = 0.0
    alignment_df[sample] = alignment_df[sample].astype('float32')
for row in tqdm(range(len(total_samp))):
    index_max = alignment_df.index.max()
    if row < range(len(total_samp))[-1]:
        if total_samp.index[row] < total_samp.index[row + 1]:
            overlap = np.where(((alignment_df.iloc[:, 0] - RT_error) <= total_samp.iloc[row, 0]) &\
                               (total_samp.iloc[row, 0] <= (alignment_df.iloc[:,0] + RT_error))\
                                 & ((alignment_df.iloc[:, 1] - mz_error) <= total_samp.iloc[row, 1])\
                                 & (total_samp.iloc[row, 1] <= (alignment_df.iloc[:, 1] + mz_error)))
            if overlap[0].size:
                overlap = [overlap][0][0][0]
                # sets intensity value into slot where realignment was identified
                alignment_df.at[overlap, alignment_df.columns[col_index]] = total_samp.iloc[row, 2]
            else:
                pass
        else:
            col_index += 1
            overlap = np.where(((alignment_df.iloc[:, 0] - RT_error) <= total_samp.iloc[row, 0]) &\
                               (total_samp.iloc[row, 0] <= (alignment_df.iloc[:,0] + RT_error))\
                                 & ((alignment_df.iloc[:, 1] - mz_error) <= total_samp.iloc[row, 1])\
                                 & (total_samp.iloc[row, 1] <= (alignment_df.iloc[:, 1] + mz_error)))
            if overlap[0].size:
                overlap = [overlap][0][0][0]
                alignment_df.at[overlap, alignment_df.columns[col_index]] = total_samp.iloc[row, 2]
            else:
                pass
    else:
        overlap = np.where(((alignment_df.iloc[:, 0] - RT_error) <= total_samp.iloc[row, 0]) &\
                            (total_samp.iloc[row, 0] <= (alignment_df.iloc[:,0] + RT_error))\
                             & ((alignment_df.iloc[:, 1] - mz_error) <= total_samp.iloc[row, 1])\
                             & (total_samp.iloc[row, 1] <= (alignment_df.iloc[:, 1] + mz_error)))
        if overlap[0].size:
            overlap = [overlap][0][0][0]
            alignment_df.at[overlap, alignment_df.columns[col_index]] = total_samp.iloc[row, 2]

        else:
            pass

alignment_df = alignment_df.fillna(0)

alignment_df = alignment_df.drop(columns=['Sum RT (min)', 'Sum Precursor m/z'])

alignment_df

100%|██████████| 3541/3541 [00:14<00:00, 245.28it/s]


,RT (min),Precursor m/z,20180815_CEC_CAL-8-no4_MSpos_1.txt,20180815_CEC_CAL-8-no4_MSpos_2.txt,20180815_CEC_CAL-8-no4_MSpos_3.txt
unique peak,,,,,
1,21.084500,102.9691,57465.140625,57533.378906,0.000000
2,0.689925,104.1072,314116.125000,403594.875000,59369.976562
3,18.046520,104.9924,0.000000,0.000000,0.000000
4,19.097570,104.9925,686955.312500,661724.937500,416176.062500
5,0.631683,104.9926,0.000000,374673.937500,648919.937500
...,...,...,...,...,...
1618,16.756830,1371.0000,0.000000,0.000000,16352.015625
1619,16.735030,1371.0080,0.000000,0.000000,0.000000
1620,16.928370,1429.0500,5220.221191,6122.378906,17692.974609
